In [22]:
#importing the necessary libraries
import cv2
from ipynb.fs.full.spot_diff import spot_diff
import time
import time
import datetime
import numpy as np
import smtplib
import imghdr
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.text import MIMEText


#We will use HOGDescriptor, i.e. Histogram of Open Gradients with SVM already implemented in OpenCV
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def find_motion():    #function that will detect if any motion is found in the video or not

	motion_detected = False
	is_start_done = False
	no_of_person = False

	cap = cv2.VideoCapture(0)  #using web camera


	check = []
	
	print("waiting for 2 seconds")
	time.sleep(2)  #waits for 2 seconds
	frame1 = cap.read()  #reading the content of video

	_, frm1 = cap.read()
	frm1 = cv2.cvtColor(frm1, cv2.COLOR_BGR2GRAY)    #converting image from one color space to another

	
	while True:
		_, frm2c = cap.read()
		frm2 = cv2.cvtColor(frm2c, cv2.COLOR_BGR2GRAY)
        
		boxes, weights = hog.detectMultiScale(frm2c, winStride=(8,8))  #
        
		boxes = np.array([[x, y, x+w, y+h] for (x, y, w, h) in boxes])
		i=0
		s_time = datetime.datetime.now()
		for(xA, yA, xB, yB) in boxes:
			cv2.rectangle(frm2c, (xA, yA), (xB, yB), (0,0,255),2) #making rectangle if human is found
			i=i+1
			cv2.putText(frm2c, 'Person: '+str(i), (xA-10, yA-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255),2)  #Text input on the output screen
			no_of_person = True  #if human detected then change no_of_person to True
#			s_time = datetime.datetime.now()



		diff = cv2.absdiff(frm1, frm2)  #calculates the difference

		_, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)   #thresholding

		contors = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

		#look at it
		contors = [c for c in contors if cv2.contourArea(c) > 25]


		if len(contors) > 5:
			cv2.putText(frm2c, "motion detected", (50,50), cv2.FONT_HERSHEY_SIMPLEX,2, (0,255,0),2)
			motion_detected = True  #if motion is detected then true
			is_start_done = False

		elif motion_detected and len(contors) < 3:
			if (is_start_done) == False:
				start = time.time()
				is_start_done = True
				end = time.time()

			end = time.time()

			print(end-start)  
			if (end - start) > 4:
				frame2 = cap.read()
				cap.release()
				cv2.destroyAllWindows()
				x = spot_diff(frame1, frame2)  #spot_diff function from spot_diff notebook
				if x == 0:  #if no difference i.e, no motion detected
					print("running again")  
					return

				else:
					print("found motion sending mail")
					return

		else:
			cv2.putText(frm2c, "no motion detected", (30,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)

		cv2.imshow("winname", frm2c)

		_, frm1 = cap.read()
		frm1 = cv2.cvtColor(frm1, cv2.COLOR_BGR2GRAY)

		count=0
#		print("time")
#		print((datetime.datetime.now().time() -s_time).total_seconds())
#		if(((datetime.datetime.now() - s_time).total_seconds)/60 < 1):

#if only there is motion and a person standing infront of door for more than a minute an alert message will be sent to house owner's mail id
		if(no_of_person == True  and motion_detected == True and ((datetime.datetime.now() - s_time).total_seconds())/60 < 1 ):
			s = smtplib.SMTP('smtp.gmail.com', 587)  #sending mail to any internet machine with an SMTP or ESMTP listener daemon
			s.starttls()  #informing email server that the email client wants to upgrade from insecure connection to a secure one using TLS or SSl
			s.login("your-email-id", "your-email-id-password")  #login in to you mail id
			msg = MIMEMultipart()  #email contains text + attachments

			cv2.imwrite("frame%d.jpg" %count, frm2c)  #save frame as jpeg
			count+=1

			with open('frame0.jpg','rb') as f:
				image_data = f.read()
                
			text = MIMEText("SECURITY ALERT!!! \n\nSomeone is outside your door")  #text message sending through mail
			msg.attach(text)
			image = MIMEImage(image_data, name='frame')  #attaching image with text in mail
			msg.attach(image)

			s.sendmail("your-email-id","the-house-owner-mail-id", msg.as_string())   #sending mail
			s.quit()  


		key = cv2.waitKey(20) & 0xFF
		if key == ord("q"):
			break
    # Press key 'q' to break the loop
            
               

	return
